In [1]:
import face_recognition
from PIL import Image
import cv2
import dlib
import numpy as np
import os

In [2]:
#通过三个点计算夹角
#b为夹角位置所在的点
#默认θ=1计算弧度，θ!不等于1时计算角度
def cos_angle(a,b,c,θ = 1):
    x,y = b-a,b-c
    Lx = np.sqrt(x.dot(x))
    Ly = np.sqrt(y.dot(y))
    cos_angle = x.dot(y)/(Lx*Ly)
    # 根据条件选择是计算弧度还是角度
    if θ != 1:
        return np.arccos(cos_angle)*360/2/np.pi
    else:
        return np.arccos(cos_angle)
    

In [3]:
#利用人脸标记模型 shape_predictor_68_face_landmarks.dat 标记人脸节点
def trait_angle(path):
    detector = dlib.get_frontal_face_detector()
    landmark_predictor = dlib.shape_predictor(
        r'shape_predictor_68_face_landmarks/shape_predictor_68_face_landmarks.dat')
    img = cv2.imread(path)
    faces = detector(img, 1)
    feas = []  # 关键点
    if (len(faces) > 0):
        for k, d in enumerate(faces):
            cv2.rectangle(img, (d.left(), d.top()), (d.right(), d.bottom()), (255, 255, 255))
            shape = landmark_predictor(img, d)

            for i in range(68):
                num = str(shape.part(i))[1:-1].split(",")
                feas.append([int(num[0]), int(num[1])])
    if not feas:
        print("List is empty")
        print(path+"不能识别")
        return
#     print('before:')
#     print(feas)
    feas = np.array(feas)
#     print('after:')
#     print(feas)
    s_fa = feas[45, :][1] - feas[36, :][1]
    a, b, c = feas[45, :], feas[36, :], np.array(feas[45, :][0], feas[36, :][1])
    if abs(s_fa) > 5:
        if s_fa > 0 and cos_angle(a, b, c,θ=4) >35:
            return cos_angle(a, b, c,θ=4)
        elif s_fa < 0 and cos_angle(a, b, c,θ=4) >35:
            return 360-cos_angle(a, b, c, θ=4)
        else:
            return 0
    else:
        return 0

In [4]:
def normalization(input,output):
    path =input
    out_path = output

    # 读取图片并识别人脸
    img = face_recognition.load_image_file(path)
    #这里如果识别不出人脸 那么len是不会大于0的 要检查好后再处理 不然报错
    encodings = face_recognition.face_encodings(img)
    if len(encodings) > 0:
        biden_encoding = encodings[0]
        face_locations = tuple(list(face_recognition.face_locations(img)[0]))
    else:
        print(path + ' 不能识别')
        return

    # 重新确定切割位置并切割
    top = face_locations[0]
    right = face_locations[1]
    bottom = face_locations[2]
    left = face_locations[3]
    cutting_position = (left, top, right, bottom)
    # 切割出人脸
    im = Image.open(path)

    region = im.crop(cutting_position)

    # 人脸缩放
    a = 50  # 人脸方格大小
    if region.size[0] >= a or region.size[1] >= a:
        region.thumbnail((a, a), Image.ANTIALIAS)
    else:
        region = region.resize((a, a), Image.ANTIALIAS)
    # 人脸旋转
    θ =trait_angle(path)
    # region = region.rotate(θ)
    # 保存人脸
    region.save(out_path)

In [5]:
#读取文件批量处理
#lfw 数据集处理
def read_img(path_lfw):
    map_path = []
    map_relative = []
    sonOfPathLfw = [y for y in os.listdir(path_lfw)]
    for i in sonOfPathLfw:
        path = path_lfw + i + '/'
        for x in os.listdir(path):
            if os.path.isfile(path + x):
                son_map_path = path + x
                map_path.append(son_map_path)
        for y in os.listdir(path):
            son_map_relative = y
            map_relative.append(son_map_relative)
    return map_path, map_relative

# 读取图片并处理保存
def read_dispose_img(path):
    map_path ,map_relative= read_img(path)
    imgs = []
    labels = []
    for idx, folder, in enumerate(map_path):
        print("读取图并处理中......"+path_normalization+map_relative[idx])
        normalization(folder,path_normalization+map_relative[idx])


In [6]:
# 数据集地址  最后的斜杆一定不要漏
path = 'D:/QIN/Face-classfication/lfw_database/lfw/'
# 处理后的数据集
path_normalization = 'D:/QIN/Face-classfication/lfw_database/normalization_lfw/'
# 模型保存地址
model_path = './人脸识别/model.ckpt'
# tfrecord文件存放路径
TFRECORD_FILE = "./人脸识别/tfrecords/"

#全局one-hot编码空间
label_binarizer = ""

In [7]:
read_img(path)

(['D:/QIN/Face-classfication/lfw_database/lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/lfw/Aaron_Guiel/Aaron_Guiel_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/lfw/Aaron_Patterson/Aaron_Patterson_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/lfw/Aaron_Peirsol/Aaron_Peirsol_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/lfw/Aaron_Peirsol/Aaron_Peirsol_0002.jpg',
  'D:/QIN/Face-classfication/lfw_database/lfw/Aaron_Peirsol/Aaron_Peirsol_0003.jpg',
  'D:/QIN/Face-classfication/lfw_database/lfw/Aaron_Peirsol/Aaron_Peirsol_0004.jpg',
  'D:/QIN/Face-classfication/lfw_database/lfw/Aaron_Pena/Aaron_Pena_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/lfw/Aaron_Sorkin/Aaron_Sorkin_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/lfw/Aaron_Sorkin/Aaron_Sorkin_0002.jpg',
  'D:/QIN/Face-classfication/lfw_database/lfw/Aaron_Tippin/Aaron_Tippin_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/lfw/Abbas_Kiarostami/Abbas_Kiarosta

In [8]:
#     处理照片
read_dispose_img(path)

读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Eckhart_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Guiel_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Patterson_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Peirsol_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Peirsol_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Peirsol_0003.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Peirsol_0004.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Pena_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Sorkin_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Sorkin_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron

读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Adoor_Gopalakarishnan_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Adriana_Lima_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Adriana_Perez_Navarro_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Adrianna_Zuzic_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Adrian_Annus_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Adrian_Fernandez_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Adrian_McPherson_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Adrian_McPherson_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Adrian_Murrell_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Adrian_Nastase_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_d

读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alan_Stonecipher_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alan_Tang_Kwong-wing_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alan_Trammell_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alan_Zemaitis_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alastair_Campbell_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alastair_Campbell_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alastair_Campbell_0003.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alastair_Campbell_0004.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alastair_Campbell_0005.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alastair_Johnston_0001.jpg
读取图并处理中......D:/QIN/Face-classficati

读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aleksander_Kwasniewski_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aleksander_Kwasniewski_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aleksander_Kwasniewski_0003.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aleksander_Kwasniewski_0004.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aleksander_Voloshin_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alek_Wek_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alessandra_Cerna_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alessandro_Nesta_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alexander_Downer_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Alexander_Downer_0002.jpg
读取图并处理中......D:/QIN/F

读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Ali_Khamenei_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Ali_Khamenei_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Ali_Khamenei_0003.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Ali_Mohammed_Maher_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Ali_Naimi_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Ali_Naimi_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Ali_Naimi_0003.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Ali_Naimi_0004.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Ali_Naimi_0005.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Ali_Naimi_0006.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Ali_Naimi_0007.jpg
读取图

...

读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Vince_Gill_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Vince_Gill_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Vince_Vaughan_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Vinnie_Jones_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Vin_Diesel_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Vin_Diesel_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Viola_Davis_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Virgina_Ruano_Pascal_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Vitali_Klitschko_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Vitali_Klitschko_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Vi

读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Wally_Szczerbiak_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Walter_Annenberg_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Walter_Mondale_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Walter_Mondale_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Walter_Mondale_0003.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Walter_Mondale_0004.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Walter_Mondale_0005.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Walter_Mondale_0006.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Walter_Mondale_0007.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Walter_Mondale_0008.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/norm

读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/William_Ford_Jr_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/William_Ford_Jr_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/William_Ford_Jr_0003.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/William_Ford_Jr_0004.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/William_Ford_Jr_0005.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/William_Ford_Jr_0006.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/William_Ford_Jr_0007.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/William_Genego_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/William_Harrison_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/William_Hochul_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database

读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Woody_Allen_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Woody_Allen_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Woody_Allen_0003.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Woody_Allen_0004.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Woody_Allen_0005.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Wu_Peng_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Wu_Yi_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Wu_Yi_0002.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Wu_Yi_0003.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Wycliffe_Grousbeck_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Xanana_Gusmao_0001.jpg
读取图并处理中.....

读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Yoriko_Kawaguchi_0007.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Yoriko_Kawaguchi_0008.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Yoriko_Kawaguchi_0009.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Yoriko_Kawaguchi_0010.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Yoriko_Kawaguchi_0011.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Yoriko_Kawaguchi_0012.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Yoriko_Kawaguchi_0013.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Yoriko_Kawaguchi_0014.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Yory_Boy_Campas_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw_database/normalization_lfw/Yoshiyuki_Kamei_0001.jpg
读取图并处理中......D:/QIN/Face-classfication/lfw

In [9]:
#测试预处理
normalization('D:/QIN/Face-classfication/lfw_database/lfw/Abdoulaye_Wade/Abdoulaye_Wade_0003.jpg','D:/QIN/Face-classfication/lfw_database/normalization_lfw/Abdoulaye_Wade_0003.jpg')

D:/QIN/Face-classfication/lfw_database/lfw/Abdoulaye_Wade/Abdoulaye_Wade_0003.jpg 不能识别


In [10]:
normalization('D:/QIN/Face-classfication/lfw_database/lfw/Binyamin_Ben-Eliezer/Binyamin_Ben-Eliezer_0001.jpg','D:/QIN/Face-classfication/lfw_database/normalization_lfw/Binyamin_Ben-Eliezer_0001.jpg')

List is empty
D:/QIN/Face-classfication/lfw_database/lfw/Binyamin_Ben-Eliezer/Binyamin_Ben-Eliezer_0001.jpg不能识别


In [11]:
normalization('D:/QIN/Face-classfication/lfw_database/lfw/Binyamin_Ben-Eliezer/Binyamin_Ben-Eliezer_0002.jpg','D:/QIN/Face-classfication/lfw_database/normalization_lfw/Binyamin_Ben-Eliezer_0002.jpg')